In [1]:
import cv2
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans

In [2]:
drawing = False
brush_size = 6
mask = None

def draw_freehand(event, x, y, flags, param):
    global drawing, mask, image

    if event == cv2.EVENT_LBUTTONDOWN:  # Start drawing
        drawing = True
        cv2.circle(mask, (x, y), brush_size, 255, -1)

    elif event == cv2.EVENT_MOUSEMOVE:  # Draw while moving the mouse
        if drawing:
            cv2.circle(mask, (x, y), brush_size, 255, -1)

    elif event == cv2.EVENT_LBUTTONUP:  # Stop drawing and apply segmentation
        drawing = False
        apply_segmentation()

def apply_segmentation():
    global image, mask

    # Extract selected region using the mask
    selected_pixels = cv2.bitwise_and(image, image, mask=mask)

    # Reshape selected region for clustering
    pixels = selected_pixels.reshape(-1, 3)
    pixels = pixels[np.any(pixels != 0, axis=1)]  # Remove black (unselected) pixels

    if pixels.size == 0:
        print("No pixels selected for segmentation.")
        return

    # # Estimate bandwidth for Mean Shift
    # bandwidth = estimate_bandwidth(pixels, quantile=0.4, n_samples=500)

    # # Apply Mean Shift clustering
    # mean_shift = MeanShift(bandwidth=5 , bin_seeding=True)
    # mean_shift.fit(pixels)

    # # Get cluster centers and labels
    # labels = mean_shift.labels_
    # cluster_centers = mean_shift.cluster_centers_

    # Define number of clusters (K)
    num_clusters = 2  # You can change this value based on your needs

    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(pixels)

    # Get cluster centers and labels
    labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_

    # Change lighter cluster centers to white
    lightness_threshold = max(list(cluster_centers.mean(axis=1)))  # Threshold for considering a color "light"
    # lightness_threshold = 60
    for i, center in enumerate(cluster_centers):
        if round(np.mean(center),2) >= round(lightness_threshold,2):  # Calculate average intensity
            cluster_centers[i] = [255, 255, 255]  # Set to white

    # Create segmented pixels
    segmented_pixels = cluster_centers[labels].astype(np.uint8)
    segmented_region = np.zeros_like(selected_pixels)
    segmented_region[np.any(selected_pixels != 0, axis=2)] = segmented_pixels

    # Add the segmented region to the main image
    white_mask = cv2.inRange(segmented_region, (255, 255, 255), (255, 255, 255))
    image[white_mask == 255] = [0, 0, 255]

    # Clear the mask for the next iteration
    mask.fill(0)


In [4]:
# Load the grayscale PNG images
images_path = '../../../Rock_CT_images/1A-135k-Data/'
import os
images = os.listdir(images_path)
images = [i for i in images if i.endswith('.png')]
images = [images_path + i for i in images]

def load_preprocess_image(image_path):
    image = cv2.imread(image_path)
    return image

image = load_preprocess_image(images[0])
# cv2.imshow("Image", image)
# key = cv2.waitKey(0)

In [5]:
for image_path in images:
    # Load the image
    # image_path = 'aligned_250.png'  # Replace with the path to your image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Unable to load image.")
        exit()

    # cv2.putText(image, f'brush_size is {brush_size}')

    # Create a black mask with the same dimensions as the image
    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    cv2.namedWindow("Image")
    cv2.setMouseCallback("Image", draw_freehand)

    while True:
        # Display the current state of the image
        display_image = cv2.addWeighted(image, 0.8, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR), 0.2, 0)
        cv2.imshow("Image", display_image)

        key = cv2.waitKey(1)

        # Press 'q' to quit
        if key == ord("q"):
            os.remove(image_path)
            cv2.imwrite('./mask/' + image_path.split('/')[-1], image)
            break
        if key == ord("r"):
            image = cv2.imread(image_path)

cv2.destroyAllWindows()


No pixels selected for segmentation.


KeyboardInterrupt: 

In [13]:
import cv2
import os
import numpy as np

# Input and output directory paths
input_dir = "./mask/"  # Replace with your image directory
output_dir = "./final_mask/"  # Replace with your desired output directory

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each image in the input directory
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        # Read the image
        image_path = os.path.join(input_dir, filename)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Failed to read {filename}, skipping.")
            continue

        # Create a binary mask: red color (0, 0, 255) -> 1, others -> 0
        mask = np.zeros(image.shape[:2], dtype=np.uint8)  # Initialize mask with 0s
        red_pixels = (image[:, :, 0] == 0) & (image[:, :, 1] == 0) & (image[:, :, 2] == 255)
        mask[red_pixels] = 1  # Set red pixels to 1

        # Save the binary mask
        mask_path = os.path.join(output_dir, os.path.splitext(filename)[0] + ".png")
        cv2.imwrite(mask_path, mask * 255)  # Scale mask to 0-255 for saving as an image
        print(f"Processed and saved mask for {filename}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: './mask/'